# 0.0. Planejamento
##  Input
### Dados: 
* **Id**: Identificador Único;
* **Title**: Título da vaga;
* **FullDescription**: Texto completo com a descrição da vaga;
* **LocationRaw**: Texto cru da localizacao da vaga;
* **LocationNormalized**: Localizacao normalizada - **Carrega erro**;
* **ContractType**: Tipo de contrato. Indica como é o regime de horas da vaga. Pode ser:
    * full_time;
    * part_time.
* **ContractTime**: Tempo de validade do contrato. Pode ser:
    * permanent;
    * contract.
* **Company**: Nome da compania que está contratanto;
* **Category**: 30 categorias de trabalho - **Carrega erro**;
* **SalaryNormalised**: Salario anualizado da vaga. O que estamos buscando prever;
* **SourceName**: Nome to site o qual recebemos a vaga.

### Problema:
* Temos um conjunto de dados com as informacoes de vagas abertas. Com base nessas \
informacoes, devemos criar um algoritmo capaz de prever o salário em novas vagas.

## Output
* **Modelo**: Um modelo capaz de prever o salario de novos usuarios;
* **WebApp**: Um webapp que carregue esse modelo e faça previsões;
* **EDA**: Uma Análise Exploratória dos Dados e do Modelo:
    - Relatório do Modelo;
    - Relatório do Projeto.


## Tasks
1. Planejamento;
2. Ciclos;
3. Construcao do Modelo Final;
    - Métrica Principal: Mean Absolute Error (MAE)

# 1.0. Imports

In [61]:
# api
import requests

# Data Manipulation
import pandas as pd
import numpy as np

# Data Plot
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import HTML
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

# Model
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from xgboost import XGBRegressor

# NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# stats
from scipy.stats import pointbiserialr, pearsonr

import pickle

[nltk_data] Downloading package punkt to /home/tadeu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tadeu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tadeu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 1.1. Helper Functions

In [62]:
def jupyter_settings():
    """
    Retorna algumas configuracoes para o jupyter notebook
    """
    %matplotlib inline
    %pylab inline
    
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container{width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()
    


def cross_val_performance(X_train, y_train, model, cv):
    """
    Retorna algumas o RMSE, MAE e MAPE em cross validation do modelo
    Input:
        X_train: os dados de treino
        y_train: A variavel resposta
        model: modelo a ser usado
        cv: numero de folds
    Output:
        Retorna um dataframe com as metricas de RMSE, MAE, MAPE e o tempo
        médio de processamento.
    """
    
    
    # Cross Val Train
    results = cross_validate(model, 
                             X_train, 
                             y_train, 
                             cv=cv,
                             scoring=[
                                 'neg_mean_absolute_error', 
                                 'neg_mean_absolute_percentage_error', 
                                 'neg_root_mean_squared_error'
                             ]
                            )
    
    # Performance
    mae = round(np.mean(results['test_neg_mean_absolute_error'] * -1), 2)
    mape = round(np.mean(results['test_neg_mean_absolute_percentage_error'] * -1), 4)
    rmse = round(np.mean(results['test_neg_root_mean_squared_error'] * -1), 2)
    time = round(np.mean(results['fit_time']), 2)
    
    df_results = pd.DataFrame(
                {
                    'Modelo': type(model).__name__,
                    'MAE': mae,
                    'MAPE': mape,
                    'RMSE': rmse,
                    'time': time
                },
                index=[0]
            )
    
    return df_results


def baseline_performance(y_train, n_splits):
    """
    Return a cross validation metric from the baseline model - mean
    Input:
        y_train: array with train response variables
        n_splits: number of folds
    """
    kf = KFold(n_splits=n_splits)
    
    mae_list = []
    mape_list = []
    rmse_list = []
    
    for train_index, valid_index in kf.split(y_train): # split index train - valid
        
        new_y_train = y_train.iloc[train_index]
        new_y_valid = y_train.iloc[valid_index]
        
        y_hat = np.repeat(new_y_train.mean(), len(new_y_valid))
        
        mae_list.append(mean_absolute_error(new_y_valid, y_hat))
        mape_list.append(mean_absolute_percentage_error(new_y_valid, y_hat))
        rmse_list.append(sqrt(mean_squared_error(new_y_valid, y_hat)))
        
    df_results = pd.DataFrame(
                {
                    'Modelo': 'Baseline',
                    'MAE': round(np.mean(mae_list), 3),
                    'MAPE': round(np.mean(mape_list), 3),
                    'RMSE': round(np.mean(rmse_list), 3),
                    'time': 0
                },
                index=[0]
            )
    
    return df_results


def tokenize(text, perm=False):
    # normalize case and remove punctuation
    if perm:
        text = re.sub(r"[^a-zA-Z0-9-]", " ", text.lower())
    else:
        text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words and remove single letters:
    tokens = [lemmatizer.lemmatize(word) for word in tokens if (word not in stop_words) and (len(word) >= 2)]

    return tokens


def comparison_continuos(df=[], names=[]):
    
    for i, dataframe in enumerate(df):
        n_df = dataframe.describe()
        n_df.loc['skew'] = dataframe.skew()
        n_df.loc['kurtosis'] = dataframe.kurtosis()
        
        n_df.name = names[i]
        
        if i == 0:
            final_df = n_df
        else:
            final_df = pd.concat([final_df, n_df], axis=1)
        
    return final_df


def city_imputation(df, column_city_raw, column_city_normalized, list_city):
    """
    Retorna uma series com as cidades extraidas da api e seu index de referencia
    Input:
        df - Dataframe usado
        column_city_raw - Coluna raw de cidades
        column_city_normalized - Coluna normaliada de cidades
        list_city = lista de cidades da api
    Output:
        Series com cada index associado a sua cidade extraida da referencia da api
        ou do coalesce com a coluna normalizada original
    """    
    
    df_aux = df.loc[:, [column_city_raw, column_city_normalized]]
    
    # For loop por cada cidade na lista da api
    for city in list_city:

        df_aux['aux'] = df_aux['LocationRaw'].apply(lambda x: city if city.lower() in x.lower() else None)
        index_val = df_aux.loc[df_aux['aux'] == city, :].index.values

        for num_index in index_val:
            dict_values[num_index] = city
            
    # Criando um DataFrame com as cidades e index
    new_city = pd.DataFrame(dict_values, index=['City']).T

    # Merge com as cidades originais
    merge_city = pd.merge(df_aux, new_city, left_index=True, right_index=True, how='left').loc[:, ['LocationNormalized', 'City']]

    # Coalesce ente as colunas
    city_series = merge_city['City'].combine_first(merge_city['LocationNormalized'])

    return city_series

In [63]:
class TransformText():
    """
    Classe usada para transformar uma coluna de textos em dummies de palavras
    e extrair informacoes relevantes como:
        1. A frequencia de cada palavra;
        2. A correlacao da palavra com a variavel resposta;
        3. O p_value dessa correlacao.
    
    Attributes:
        dataframe (dataframe pandas) - dataframe a ser utilizado para treino;
        text_col (str) - nome da coluna contendo texto;
        response_col (str) - nome da coluna com a variavle resposta continua.
        
    OBS:
        Por ser um protótipo, essa classe segue alguns preceitos desse projeto 
        especifico:
            1. Necessita de uma variavel resposta;
            2. Essa variavel resposta tem que ser continua.
    """
    
    def __init__(self, df, text_col='', response_col=''):
        
        self.text_name = text_col
        self.response_name = response_col
        self.dataframe = df.loc[:, [self.text_name, self.response_name]]
    

    def unique_tokenize(self, text):
        """
        Funcao utilizada para aplicar o processo de tokenizacao de um texto
        Input:
            text: Texto que sera tokenizado
        Output:
            uma lista de palavras que passam pelas seguintes etapas:
                1. Sao selecionados apenas valores a-zA-Z0-9;
                2. Todo é colocado em minusculo;
                3. O texto é tokenizado com word_tokenize;
                4. o texto é lemmatizado e as stop_words sao retiradas;
                5. Apenas valores com tamanho maior que 2 sao selecionados.
        """
        
        # normalize case and remove punctuation
        text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

        # tokenize text
        tokens = word_tokenize(text)

        # lemmatize andremove stop words and remove single letters:
        tokens = [
            lemmatizer.lemmatize(word) 
            for word 
            in tokens 
            if (word not in stop_words) 
            and (len(word) >= 2)
        ]
        
        return tokens
        
        
    def tokenize(self):
        """
        Funcao utilizada para aplicar a funcao unique_tokenize em todas as 
        linhas de text_name e guardar em self.tokens
        """
        self.tokens = self.dataframe[self.text_name].apply(lambda x: self.unique_tokenize(x))
        
        return self.tokens
    
    
    def get_frequency(self):
        """
        Retorna a frequencia absoluta de cada palavra em text_name
        """
        # gerando o dataframe com as contagens
        final_df = self.tokenize().explode().value_counts().reset_index().rename({'index': 'Word',
                                                                                  self.text_name: 'Count'},
                                                                                 axis=1)
        
        self.frequency_ = final_df.sort_values(by='Count', ascending=False)
        
        return self.frequency_

    
    def unique_words(self, sample_size):
        """
        Gera uma lsita das palavras unicas de acordo com o tamanho desejado
        Input:
            sample_size (int64) - quantas linhas quero retornar.
        Output:
            Retorna uma lista com os valores unicos mais frequentes na ordem
            de grandeza de sample_size.
        """        
        
        return self.get_frequency().head(sample_size)['Word'].unique()
        
    

    def get_corr(self, sample_size):
        """
        Faz o calculo de correlacao utilizando o metodo pointbiserial
        Input:
            sample_size (int64) - sample_size para contar o numero de palavras
            desejadas em unique_words
        Output:
            Um DataFrame com as seguintes colunas:
                Word - A palavra utilizada;
                Corr - A correlacao entre a palavra e a variável resposta;
                p-value - o p-value dessa correlacao.
        """
        dict_words = {} # dicionario a ser alimentado
        
        list_words = self.unique_words(sample_size) # chamando a funcao com limite de palavras
        array_x = self.tokens # determinadno o array com os tokens
        array_y = self.dataframe[self.response_name].values # determinando o array com a var resposta
        
        for word in list_words: # iterando por cada palavra selecionada
            
            # se a palavra estiver no texto, atribui-se 1, senao atribui-se 0
            list_values = [1 if word in row else 0 for row in array_x] 
            # calculando a correlacao através do método do ponto bisserial
            result = pointbiserialr(list_values, self.dataframe[self.response_name].values)
            
            dict_words[word] = [result[0], result[1]]
            
        self.corr_ = pd.DataFrame(dict_words, index=['Corr', 'p-value']).T.sort_values(by='Corr')

        return self.corr_


    def transform(self, sample_size, threshold=0.0):
        """
        gera um DataFrame com cada palavra sendo uma coluna sendo que:
            1 -> a palavra esta no texto;
            2 -> a palavra nao esta no texto.
        Input:
            sample_size (int64) - o numero de palavras em unique_words;
            theshold (float64) - a correlacao minima de uma palavra para ser considerada.
        """

        # transformando as correlacoes em valores absolutos
        df_cor = self.get_corr(sample_size).apply(lambda x: abs(x))

        # Colunas que serao usadas
        selected_cols = df_cor.loc[df_cor['Corr'] > threshold, :].index.to_list()
        
        self.list_words = selected_cols
        
        df_text = pd.DataFrame(self.tokens)
        
        # imputando as colunas
        for col in selected_cols:
            coluna = pd.Series(df_text[self.text_name].apply(lambda x: 1 if col in x else 0,), name=col,index=df_text.index.to_list())
            df_text = pd.concat([df_text, coluna], axis=1)
            
        self.dummy_cols_ = df_text.drop([self.text_name], axis=1)
        
    
    def get_all(self):
        """
        Retorna:
            df_corr_freq: dataframe com a frequencia, correlacao e p_value de cada palavra;
            dummy_cols: dataframe com as colunas dummy.
        """
        
        df_corr_freq = pd.merge(self.frequency_, self.corr_, left_on='Word', right_index=True)
        
        return df_corr_freq, self.dummy_cols_

In [64]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 1.2. Data

In [65]:
data_import = pd.read_csv('../data/Train_rev1.csv')

data_import.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


# 2.0. Data Description

In [66]:
df2 = data_import.copy()

## 2.1. Shape Data

In [67]:
print(f'Rows: {df2.shape[0]}')
print(f'Columns: {df2.shape[1]}')

Rows: 244768
Columns: 12


## 2.2. Data Types

In [68]:
df2.dtypes

Id                     int64
Title                 object
FullDescription       object
LocationRaw           object
LocationNormalized    object
ContractType          object
ContractTime          object
Company               object
Category              object
SalaryRaw             object
SalaryNormalized       int64
SourceName            object
dtype: object

## 2.3. Check Na


In [69]:
tot_na = df2.isnull().sum()
percentage_na = round(df2.isnull().mean(), 4) * 100

# dataframe of na
pd.DataFrame([tot_na, percentage_na], index=['Total', 'Percentual']).T

,Total,Percentual
Id,0.0,0.00
Title,1.0,0.00
FullDescription,0.0,0.00
LocationRaw,0.0,0.00
LocationNormalized,0.0,0.00
ContractType,179326.0,73.26
ContractTime,63905.0,26.11
Company,32430.0,13.25
Category,0.0,0.00
SalaryRaw,0.0,0.00


# 3.0. Variable Filtering + Split

In [70]:
df3 = df2.copy()

In [71]:
df3.drop(['Id', 'SalaryRaw', 'SourceName', 'Company'], axis=1, inplace=True)
df3.dropna(subset=['Title'], axis=0, inplace=True)

## 3.1. Split Dataset

In [72]:
y = df3.loc[:, 'SalaryNormalized']
X = df3.drop('SalaryNormalized', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4.0. Feature Engineering

In [73]:
# X_train merge with y
X_train = pd.merge(X_train, y_train, right_index=True, left_index=True)

In [74]:
# list de stop_words
stop_words = stopwords.words("english")

# instanciando o lemmatizer
lemmatizer = WordNetLemmatizer()

## 4.1. Title - Salvo

In [ ]:
# instanciando a classe TransformText()
transf_title = TransformText(df=X_train, text_col='Title', response_col='SalaryNormalized')

# transformando as colunas
transf_title.transform(sample_size=2500, threshold=0.005)

# recebendo as informacoes: Count, Corr e p-value | colunas dummys
df_infos_title, dummy_cols_title = transf_title.get_all()

In [ ]:
# save df's in csv file
dummy_cols_title.to_csv('../data/dummy_cols_title_3.csv')
#df_infos_title.to_csv('../data/df_infos_title.csv')

## 4.2. Full Description - Salvo & Retirado C03

In [ ]:
## instanciando a classe TransformText()
#transf_desc = TransformText(df=X_train, text_col='FullDescription', response_col='SalaryNormalized')
#
## transformando as colunas
#transf_desc.transform(sample_size=2000, threshold=0.02)
#
## recebendo as informacoes: Count, Corr e p-value | colunas dummys
#df_infos_desc, dummy_cols_desc = transf_desc.get_all()

In [ ]:
## save df's in csv file
#df_infos_desc.to_csv('../data/df_infos_desc.csv')
#dummy_cols_desc.to_csv('../data/dummy_cols_desc.csv')

## 4.3. SourceName - Retirado C04

In [ ]:
#a = X_train['SourceName']
#
#array_y = y_train.values
#
#dict_results = {}
#
#for column in source_dummies.columns:
#    array_x = source_dummies[column].values
#    
#    result = pointbiserialr(array_x, array_y)
#    
#    dict_results[column] = [result[0], result[1]]
#    
#source_dummies = pd.get_dummies(a)
#
#df = pd.DataFrame(dict_results, index=['Corr', 'p-value']).T
#
#df['Corr_abs'] = df['Corr'].apply(lambda x: abs(x))
#
#df_sorted = df.sort_values('Corr_abs', ascending=False)
#
#dummies_source = source_dummies.loc[:, df_sorted[df_sorted['Corr_abs'] >= 0.15].index]

## 4.4. Company - Retirado C04

In [ ]:
#dum = pd.get_dummies(X_train['Company'])
#
#array_y = y_train.values
#
#dict_results = {}
#
#for column in dum.columns:
#    array_x = dum[column].values
#    
#    result = pointbiserialr(array_x, array_y)
#    
#    dict_results[column] = [result[0], result[1]]
#    
#df = pd.DataFrame(dict_results, index=['Corr', 'p-value']).T

## 4.5. Location

In [ ]:
# Gerando um DataFrame auxiliar
dummies_city = X_train.loc[:, ['LocationRaw', 'LocationNormalized']]

In [ ]:
# Fazendo uma requisicao na api para pegar as cidades de UK

url = 'https://countriesnow.space/api/v0.1/countries/cities'
myobj = {
    "country": "United Kingdom"
}

x = requests.post(url, data = myobj)

In [ ]:
# Lista das cidades da API
city_api_list = pd.Series(x.json()['data']).to_list()

# Criando um Dicionario que associa o index com cada cidade
dict_values = {}

In [ ]:
#column_new_city = city_imputation(X_train, 'LocationRaw', 'LocationNormalized', city_api_list)
#
## Salvando a series
#column_new_city.to_csv('../data/new_location.csv')

In [ ]:
# Importando o csv
columns_city = pd.read_csv('../data/new_location.csv', index_col=0)

# Merge com os dados originais
X_train = pd.merge(X_train, columns_city, left_index=True, right_index=True)

# Excluindo a coluna original
X_train = X_train.drop('LocationNormalized', axis=1)

In [ ]:
# Atribuindo nomes a Londres
X_train['City'] = X_train['City'].apply(lambda x: 'London' if 'London' in x else x)

In [ ]:
# Teste de correlacao
dum = pd.get_dummies(X_train['City'])

array_y = y_train.values

dict_results = {}

for column in dum.columns:
    array_x = dum[column].values
    
    result = pointbiserialr(array_x, array_y)
    
    dict_results[column] = [result[0], result[1]]
    
df = pd.DataFrame(dict_results, index=['Corr', 'p-value']).T

In [ ]:
df_corr = df.sort_values(by='Corr', ascending=False)

In [ ]:
list_corr_city = df_corr.head(5).index.to_list()

In [ ]:
df_aux = X_train.loc[:, ['City', 'LocationRaw']]

for column in list_corr_city:
    df_aux[column] = df_aux['City'].apply(lambda x: 1 if column == x else 0)

dummies_city = df_aux.drop(['City', 'LocationRaw'], axis=1)

# 5.0. Exploratory Data Analysis

In [ ]:
color_blue = '#5975a4'
color_red = '#CD5C5C'

## 5.1. Response Variable - SalaryNormalized

### 5.1.1. Distribuicao

A Variável Resposta é **Distribuicao Normal Assimétrica a Direita**
- Média > Mediana

In [ ]:
comparison_continuos([y_train], ['Normal'])

In [ ]:
fig, ax = plt.subplots(2, 1)

# plot histogram
sns.histplot(y_train, ax=ax[0], bins=100)

# plot boxplot
sns.boxplot(x=y_train, ax=ax[1])

# title
ax[0].title.set_text('Distribuicao Da Variável Salário')

fig.tight_layout()

### 5.1.2. Outliers

**Método do Intervalo Interquartil para a idenficicacao de Outliers:**   

IQR = Q3 - Q1  

High Outlier = Q3 + (1.5 * IQR)  

**Limite Superior**: 74.000

**Dois testes a serem Feitos em Pŕoximo Ciclo:**  
    * Retirar valores acima de 100k  
    * Retirar valores acima de 74k  

In [ ]:
# Calculo de outliers superiores - Nossos dados nao possuem outliers inferiores
high_outlier = y_train.quantile([0.75]) + (y_train.quantile([0.75]).values - y_train.quantile([0.25]).values) * 1.5

print(f'Limite Superior - Outliers: {high_outlier.values[0]}')

In [ ]:
y_outliers = y_train[y_train > 74000]

print(f'% de Entradas maiores que 74.000: {round((len(y_outliers) / len(y_train) * 100), 2)}%')
print(f'% de Entradas maiores que 100.000: {round((len(y_outliers[y_outliers>100000]) / len(y_train) * 100), 2)}%')

In [ ]:
y_outliers.hist(bins=100)

## 5.2. ContractType

**Null Values**:
* Os valores de NA possuem uma distribuicao bem semelhante ao Contract_Full:
    * Podemos ajustar a premissa de que se na descricao nao se especifica   
    o tipo de contrato, está implítico que é full_time.
    * Realizar um teste de hipotese para confirmar isso no próximo ciclo.
    
**Distribuicao**:
* Como é de se esperar, contratos de meio período indicam um menor salario

In [ ]:
# Select data
contract_null = df2.loc[df2['ContractType'].isnull(), 'SalaryNormalized']
contract_full = df2.loc[df2['ContractType'] == 'full_time', 'SalaryNormalized']            
contract_part = df2.loc[df2['ContractType'] == 'part_time', 'SalaryNormalized']      

In [ ]:
comparison_continuos([y_train, contract_null, contract_full, contract_part], 
                     ['Normal', 'Contract_null', 'Contract_full', 'Contract_part'])

In [ ]:
# plot
sns.countplot(y=X_train['ContractType'].apply(lambda x: str(x)), 
              order=['nan', 'full_time', 'part_time'], 
              color=color_blue)

# title
plt.title('Contagem de Valores ContractType')

In [ ]:
fig, ax = plt.subplots(3, 1)

# plots
sns.boxplot(x=contract_null, ax=ax[0])
sns.boxplot(x=contract_full, ax=ax[1])
sns.boxplot(x=contract_part, ax=ax[2])

# Titles
ax[0].title.set_text('Null Values')
ax[1].title.set_text('full_time')
ax[2].title.set_text('part_time')

# Colocando os eixos em mesma ordem
for value in range(0, 3):
    ax[value].set_xlim([0, 210000])

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2)

# plots
sns.histplot(x=contract_null, ax=ax[0], bins=100)
sns.histplot(x=contract_full, ax=ax[1], bins=100)

# title
ax[0].title.set_text('Null Values Salary')
ax[1].title.set_text('Full Time Salary')

## 5.3. ContractTime
**Null Values**:
* Um quantidade consideravel de Null Values;

**Distribuicao**:
* A distribuicao dos 3 tipos sao bem semelhantes

In [ ]:
# select data
null = df2.loc[df2['ContractTime'].isnull(), 'SalaryNormalized']
contract = df2.loc[df2['ContractTime'] == 'contract', 'SalaryNormalized']            
permanent =  df2.loc[df2['ContractTime'] == 'permanent', 'SalaryNormalized']      

In [ ]:
comparison_continuos([y_train, null, contract, contract], 
                     ['Normal', 'Null', 'Contract', 'Contract'])

In [ ]:
# plot
sns.countplot(y=X_train['ContractTime'].apply(lambda x: str(x)), 
              order=['permanent', 'nan', 'contract'], 
              color=color_blue)

# title
plt.title('Contagem de Valores ContractType')

In [ ]:
fig, ax = plt.subplots(3, 1)

# plots
sns.boxplot(x=null, ax=ax[0])
sns.boxplot(x=contract, ax=ax[1])
sns.boxplot(x=permanent, ax=ax[2])

# Titles
ax[0].title.set_text('Null Values')
ax[1].title.set_text('Contrato')
ax[2].title.set_text('Permanente')

# Colocando os eixos em mesma ordem
for value in range(0, 3):
    ax[value].set_xlim([0, 210000])

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 3)

# plots
sns.histplot(x=null, ax=ax[0], bins=100)
sns.histplot(x=contract, ax=ax[1], bins=100)
sns.histplot(x=permanent, ax=ax[2], bins=100)

# title
ax[0].title.set_text('Null Values Salary')
ax[1].title.set_text('Contract Salary')
ax[2].title.set_text('Permanent Salary')

## 5.4. Title

In [ ]:
# Import infos
title_info = pd.read_csv('../data/df_infos_title.csv').drop('Unnamed: 0', axis=1)

# DataFrames Auxiliares
title_corr = title_info[['Word', 'Corr']].sort_values(by='Corr', ascending=False)
title_freq = title_info[['Word', 'Count']].sort_values(by='Count', ascending=False)

# Dicionario de contagem para a nuvem de palavras
dict_freq_title = title_freq.set_index('Word').to_dict()['Count']

In [ ]:
# geranndo os subplots
fig, ax = plt.subplots(1, 2)

# setanto titulos
ax[0].title.set_text('Corr Negativa')
ax[1].title.set_text('Corr Positiva')

# plots
sns.barplot(x='Corr', 
            y='Word', 
            data=title_corr.tail(10).sort_values(by='Corr', ascending=True), 
            color=color_red, ax=ax[0])

sns.barplot(x='Corr', 
            y='Word', 
            data=title_corr.head(10), 
            color=color_blue, ax=ax[1])

In [ ]:
# geranndo os subplots
fig, ax = plt.subplots(1, 2)

# setanto titulos
ax[0].title.set_text('Frequencia de Palavras')
ax[1].title.set_text('Nuvem de Palavras')

# plots
sns.barplot(x='Count', 
            y='Word', 
            data=title_freq.head(10).sort_values(by='Count', ascending=False), 
            color=color_blue, ax=ax[0])


# Nuvem de palavras
# Color mask
mask = np.array(Image.open("../img/blue_palette.png"))

# Instanciando a color mark
image_colors = ImageColorGenerator(mask)

# Configurando a WordCloud
wc = WordCloud(background_color="white", max_words=100,
               max_font_size=256, mode='RGBA',
               random_state=42, width=500, height=500)

# Plot na axis
ax[1] = wc.generate_from_frequencies(dict_freq_title)
imshow(wc.recolor(color_func=image_colors), interpolation="bilinear")
axis('off')

## 5.5. Describe

In [ ]:
# Import infos
desc_info = pd.read_csv('../data/df_infos_desc.csv').drop('Unnamed: 0', axis=1)

# DataFrames Auxiliares
desc_corr = desc_info[['Word', 'Corr']].sort_values(by='Corr', ascending=False)
desc_freq = desc_info[['Word', 'Count']].sort_values(by='Count', ascending=False)

# Dicionario de contagem para a nuvem de palavras
dict_freq_desc = desc_freq.set_index('Word').to_dict()['Count']

In [ ]:
# geranndo os subplots
fig, ax = plt.subplots(1, 2)

# setanto titulos
ax[0].title.set_text('Corr Negativa')
ax[1].title.set_text('Corr Positiva')

# plots
sns.barplot(x='Corr', 
            y='Word', 
            data=desc_corr.tail(10).sort_values(by='Corr', ascending=True), 
            color=color_red, ax=ax[0])

sns.barplot(x='Corr', 
            y='Word', 
            data=desc_corr.head(10), 
            color=color_blue, ax=ax[1])

In [ ]:
# geranndo os subplots
fig, ax = plt.subplots(1, 2)

# setanto titulos
ax[0].title.set_text('Frequencia de Palavras')
ax[1].title.set_text('Nuvem de Palavras')

# plots
sns.barplot(x='Count', 
            y='Word', 
            data=desc_freq.head(10).sort_values(by='Count', ascending=False), 
            color=color_blue, ax=ax[0])


# Nuvem de palavras
# Color mask
mask = np.array(Image.open("../img/blue_palette.png"))

# Instanciando a color mark
image_colors = ImageColorGenerator(mask)

# Configurando a WordCloud
wc = WordCloud(background_color="white", max_words=100,
               max_font_size=256, mode='RGBA',
               random_state=42, width=500, height=500)

# Plot na axis
ax[1] = wc.generate_from_frequencies(dict_freq_desc)
imshow(wc.recolor(color_func=image_colors), interpolation="bilinear")
axis('off')

# 6.0. Data Preparation

In [ ]:
X_train_6 = X_train.copy()

## 6.1. Filter

In [ ]:
X_train_6 = X_train.drop(['SalaryNormalized', 'Title', 'FullDescription', 'LocationRaw', 'City'], axis=1)

## 6.2. NA Values

In [ ]:
X_train_6.loc[X_train['ContractType'].isnull(), 'ContractType'] = 'full_time'
X_train_6.loc[X_train['ContractTime'].isnull(), 'ContractTime'] = 'permanent'

## 6.3. Categorical

### 6.3.1. One Hot Encoder

In [ ]:
trf = ColumnTransformer(transformers =[
    ('enc', OneHotEncoder(sparse = False, drop ='first'), list(range(3))),
], remainder ='passthrough')

In [ ]:
X_train_6 = pd.DataFrame(trf.fit_transform(X_train_6))

### 6.2.3. Join Dummies

In [ ]:
# Import Dummy Columns
dummy_cols_title = pd.read_csv('../data/dummy_cols_title_3.csv', index_col=0).reset_index(drop=True)

# 7.0. Feature Selection

In [ ]:
X_train_t = pd.concat([X_train_6, dummy_cols_title.reset_index(drop=True), dummies_city.reset_index(drop=True)], axis=1)

In [ ]:
X_train_t.columns = [value for value in range(1, len(X_train_t.columns) + 1)]

# 8.0. Machine Learning Pipeline

## 8.1. Baseline - Mean

In [ ]:
base = baseline_performance(y_train, 5)

## 8.2. Linear Regression

In [ ]:
lr = LinearRegression()

lr_model = cross_val_performance(X_train_t, y_train, cv=5, model=lr)

In [ ]:
lr_model

## 8.3. DecisionTreeRegressor

In [ ]:
dtr = DecisionTreeRegressor()

dtr_model = cross_val_performance(X_train_t, y_train, cv=5, model=dtr)

In [ ]:
dtr_model

## 8.4. Comparance

In [ ]:
cross_performance = pd.concat([base, lr_model, dtr_model]).sort_values(by='MAE').reset_index(drop=True)

cross_performance

In [ ]:
# save performance
cross_performance.to_csv('../data/cross_performance_c05_2.csv')

# 9.0. Test Data

In [ ]:
class test_evaluation(dataset, columns_title, city_api_list):
    def __init__():
        self.dataset = dataset
        self.dummy_cols_title = columns_title
        self.city_api_list = city_api_list
    
    def select_data(self):
        X_test = dataset.loc[:, 
                            ['Title', 'LocationRaw', 
                            'LocationNormalized', 'ContractType', 
                            'ContractTime', 'Category']
                           ]
        self.dataset = X_test
        
    
    def one_hot(self):
        # selecionando as 3 colunas iniciais
        dums = self.dataset.loc[:, ['ContractType', 'ContractTime', 'Category']]

        # Filtrando null values
        dums.loc[dums['ContractType'].isnull(), 'ContractType'] = 'full_time'
        dums.loc[dums['ContractTime'].isnull(), 'ContractTime'] = 'permanent'

        # Transformando as colunas em binarias
        trf = ColumnTransformer(transformers =[
            ('enc', OneHotEncoder(sparse = False, drop ='first'), list(range(3))),
        ], remainder ='passthrough')

        dums = pd.DataFrame(trf.fit_transform(dums))
        
        self.one_hot_ccc = dums
        
    
    def concat_data(self):
        title_corr_final = [word for word in dummy_cols_title.columns]
        
        X_test_t = pd.concat([self.dataset.loc[:, 
                                         ['Title', 
                                          'LocationRaw', 
                                          'LocationNormalized']].reset_index(drop=True), 
                              dums], axis=1)
        
        self.dataset = X_test_t
        
    
    def title_transform(self):
        # Tokenizando
        self.dataset['Title'] = self.dataset['Title'].apply(lambda x: tokenize(x))

        # Transformando os tokens
        for value in self.dummy_cols_title:
            self.dataset[value] = self.dataset['Title'].apply(lambda x: 1 if value in x else 0)
    
    
    def city_transform(self):
        # city
        column_new_city = city_imputation(X_test, 'LocationRaw', 
                                          'LocationNormalized', 
                                          self.city_api_list)
        
        self.column_city = column_new_city
        
        
    def concat_all(self):
        self.dataset = pd.concat([self.dataset, self.column_city.reset_index(drop='True')], axis=1)
        
        
    def final_preparation(self):
        df_aux = X_test_t.loc[:, ['City', 'LocationRaw']]

        for column in list_corr_city:
            df_aux[column] = df_aux['City'].apply(lambda x: 1 if column == x else 0)

        dummies_city = df_aux.drop(['City', 'LocationRaw'], axis=1)

In [ ]:
title_corr_final = [word for word in dummy_cols_title.columns]

In [ ]:
X_test_t = pd.concat([X_test.loc[:, ['Title', 'LocationRaw', 'LocationNormalized']].reset_index(drop=True), dums], axis=1)

In [ ]:
# Tokenizando
X_test_t['Title'] = X_test_t['Title'].apply(lambda x: tokenize(x))

# Transformando os tokens
for value in title_corr_final:
    X_test_t[value] = X_test_t['Title'].apply(lambda x: 1 if value in x else 0)

In [ ]:
# city
column_new_city = city_imputation(X_test, 'LocationRaw', 'LocationNormalized', city_api_list)

In [ ]:
X_test_t = pd.concat([X_test_t, column_new_city.reset_index(drop='True')], axis=1)

In [ ]:
df_aux = X_test_t.loc[:, ['City', 'LocationRaw']]

for column in list_corr_city:
    df_aux[column] = df_aux['City'].apply(lambda x: 1 if column == x else 0)

dummies_city = df_aux.drop(['City', 'LocationRaw'], axis=1)

In [ ]:
X_test_t = pd.concat([X_test_t, dummies_city], axis=1)

In [ ]:
X_test_t = X_test_t.drop(['City', 'LocationRaw', 'LocationNormalized', 'Title'], axis=1)

In [ ]:
X_test_t.columns = [value for value in range(1, len(X_test_t.columns) + 1)]

In [ ]:
dtr = DecisionTreeRegressor()

dtr.fit(X_train_t, y_train)

In [ ]:
y_hat = dtr.predict(X_test_t)

In [ ]:
mean_absolute_error(y_test, y_hat)

# 10.0. Deploy